In [ ]:
from util.lexicon_dataloader import LexiconDataLoader
import pandas as pd

In [2]:
LEXICON_PATH = "data/VnEmoLex.xlsx"

In [4]:
lexicon = LexiconDataLoader(LEXICON_PATH)

In [ ]:
lexicon_dataframe = lexicon.data.DataFrame